# #412 Multi span concrete slab bridge deck with integral substructures
<i>Modelled in shell elements</i>
***

In [ ]:
solve_analyses = False      # Solve after script finishes
dmi_grid = False            # Use nodes in search area instead
influence_locations = [0.5] #[0.0, 0.2, 0.4, 0.6, 0.8, 1.0]  # Parametric distances along each line
influence_lines_y   = [3]   #[2,3,4]                         # Lines across the deck to consider

Define model parameters

In [ ]:
# Spans
span_lengths = [15, 20, 15]
skew_angle = 25

# Geometric
deck_width = 11
deck_overhang = 1.5
deck_thk = 1.0
deck_thk_outstand = 0.25
# deck_thk_pier = 1.4

abut_thk = 1.0
pier_thk = 1.1
# pier_head = 2.0

pier_height = 5


# Meshing
mesh_size = 2

# Loading area
carriageway_width = 11

# Loads
surf_load_intensity = -2.5
parapet_load_udl = -2.5
footpath_surface_load_intensity = -1 # Additional thickness of footpath
settlement = -0.01 # m

# Temperatures
temperature_increase = 20
temperature_decrease = -20
temp_max = 25
temp_min = 0.0

# Wind load
wind_udl = 10
wind_mom = 2

# Live loads
pedestrian_live_load = -5 # kpa
breaking_load = 900 # kN
horiz_load = 200    # kN
pier_impact_load = 1000 # kN
deck_impact_load = 1000 # kN/m

In [ ]:
# Calculated dimensions
overall_length = sum(span_lengths) + abut_thk
overall_width  = 2*deck_overhang + deck_width

footway_width = (overall_width-carriageway_width)/2

breaking_load_intensity = breaking_load / (carriageway_width*overall_length)

# eccentcity_pier = (deck_thk_pier-deck_thk)/2
eccentcity_outstand = (deck_thk_outstand-deck_thk)/2

Connect to LUSAS and either create a new model or delete the contents of the existing one - be careful if you have a model already loaded

In [ ]:
import numpy as np
import math
import sys; sys.path.append('../') # Reference modules in parent directory
from LPI import *
lusas = get_lusas_modeller()

from m100_Tools_And_Helpers import Helpers
Helpers.initialise(lusas)

# if lusas.existsDatabase():
#     raise Exception("This script will create a new model. Please save and close the current model and try again")

# Create a new model
lusas.newProject("Structural", f"Integral Slab Bridge Deck")
# Reference to the model database for convenience
db = lusas.database() 
# 3D model with Z vertical
db.setAnalysisCategory("3D")
db.setVerticalDir("Z")
# Set the unit system
db.setModelUnits("kN,m,t,s,C")

Create the attributes to be assigned to the geometric features. Mesh, Geometric and Material.

In [ ]:
# Concrete material attribute
concrete_mat_attr = db.createIsotropicMaterial("Concrete", 34.8E6, 0.2, 2.4, 10e-6)

# Set the Concrete material to be default and automatically assigned to all newly created geometry 
db.setAsDefault("Material", concrete_mat_attr)

# Deck mesh attribute, thick shell elements
deck_mesh_attr         = db.createMeshSurface("SMsh1").setRegularSize("QTS4", mesh_size, True)
# Deck surface thickness attribute
deck_surface_attr      = db.createGeometricSurface(f"Deck Slab {deck_thk:.1f}m").setSurface(deck_thk, 0.0)
outstand_surface_attr  = db.createGeometricSurface(f"Deck Outstand {deck_thk_outstand:.1f}m").setSurface(deck_thk_outstand, eccentcity_outstand)
abut_surface_attr      = db.createGeometricSurface(f'Abutment {abut_thk:.1f}m').setSurface(abut_thk)
pier_surface_attr      = db.createGeometricSurface(f'Pier {pier_thk:.1f}m').setSurface(pier_thk)

# Support attributes
support_pinned_attr = db.createSupportStructural("Pinned").setStructural("R", "R", "R", "F", "F", "F", "F", "F", "C", "F") # F=Free, R=Restrained
support_slide_attr  = db.createSupportStructural("Slide").setStructural("F", "R", "R", "F", "F", "F", "F", "F", "C", "F")
support_fixed_attr  = db.createSupportStructural("Fully Fixed").setStructural("R", "R", "R", "R", "R", "R", "F", "F", "C", "F")

In [ ]:
conc_creep_attr = db.createIsotropicMaterial("Concrete Creep & Shrinkage", 36.75E6, 0.2, 2.5484)
conc_creep_attr.addEurocodePlasticMaterial86(0.0, 0.0)
conc_creep_attr.addEurocode2ConcreteCreep("48.0E3", "class N", "70.0", 0.0, False, 1.0, True, 1, 0.0)
conc_creep_attr.addEurocode2ConcreteShrinkage(True)


In [ ]:
# Local coords for skew angle
skew_coords_attr = db.createLocalCartesianXYAttr("Skew", -skew_angle, [0,0,0]).setAxesType("Cartesian")
skew_coords_attr.visualiseDefn(False)

In [ ]:
# Helper function to calculate beam coordinates accounting for the skew angle
def skew_x(y:float)->float:
    return y * math.tan(math.radians(skew_angle))

## Create the model geometry

In [ ]:
# X Coordinates for the entire deck grid
x_grid = list()
x = abut_thk/2
# First support
x_grid.extend([0.0, x, x+abut_thk/2])
# Pier lines
for ix in range(0, len(span_lengths)-1):
    x+=span_lengths[ix]
    x_grid.extend([x-pier_thk/2, x, x+pier_thk/2])
# final support
x+=span_lengths[-1]
x_grid.extend([x-abut_thk/2, x, x+abut_thk/2])

# Y Coordinates for deck grid
y_grid = [-deck_width/2-deck_overhang, -deck_width/2, -deck_width/3, 0.0, deck_width/3, deck_width/2, deck_width/2+deck_overhang]

In [ ]:
# Create the actual points in the model [X][Y]
deck_points = [[Helpers.create_point(x+skew_x(y), y, 0.0) for y in y_grid] for x in x_grid]

In [ ]:
''' Create Deck surfaces '''
# Set the default mesh attribute which is automatically assigned to each newly created surface
db.setAsDefault("Surface Mesh", deck_mesh_attr)
# Set a default group to which the new geometry is added
group = db.createGroup("Deck").setCurrent()

# Create the surfaces for the first outstand
db.setAsDefault("Surface Geometric", outstand_surface_attr)
for ix in range(0, len(x_grid)-1):
    Helpers.create_surface_from_points([ deck_points[ix][0], deck_points[ix+1][0], deck_points[ix+1][1], deck_points[ix][1] ])

# Create the surfaces for the main deck
db.setAsDefault("Surface Geometric", deck_surface_attr)
for iy in range(1, len(y_grid)-2):
    for ix in range(0, len(x_grid)-1):
        Helpers.create_surface_from_points([ deck_points[ix][iy], deck_points[ix+1][iy], deck_points[ix+1][iy+1], deck_points[ix][iy+1] ])

# Create the surfaces for the second outstand
db.setAsDefault("Surface Geometric", outstand_surface_attr)
for ix in range(0, len(x_grid)-1):
    Helpers.create_surface_from_points([ deck_points[ix][-2], deck_points[ix+1][-2], deck_points[ix+1][-1], deck_points[ix][-1] ])

db.setCurrent() # remove the default group

In [ ]:
# Determine the array indices for sub structure locations
support_indices = [1]
for s in span_lengths:
    support_indices.append(support_indices[-1]+3)

# Sub structure points
base_points = {}
for ix in support_indices:
    base_points[ix] = [Helpers.create_point(p.getX(), p.getY(), p.getZ() - pier_height) for p in deck_points[ix]]

In [ ]:
# Sub structure surfaces
ia = 1
ip = 1
for ix in support_indices:
    isAbut = (ix == support_indices[0] or ix == support_indices[-1])
    if isAbut:
        group = db.createGroup(f"Abut {ia}") ; ia+=1
        db.setAsDefault("Surface Geometric", abut_surface_attr)
    else:
        group = db.createGroup(f"Pier {ip}") ; ip+=1
        db.setAsDefault("Surface Geometric", pier_surface_attr)

    for iy in range(1, len(y_grid)-2):
        s = Helpers.create_surface_from_points([deck_points[ix][iy], deck_points[ix][iy+1], base_points[ix][iy+1], base_points[ix][iy]])
        group.add(s) 

In [ ]:
# Supports
for ix in support_indices:
    points = base_points[ix]
    for iy in range(1, len(y_grid)-2):
        line = Helpers.get_line_between_points(base_points[ix][iy], base_points[ix][iy+1])
        support_fixed_attr.assignTo(line)
        skew_coords_attr.assignTo(line)

In [ ]:
# Turn off default attribute assignments
db.setAsDefault("Mesh", "None")
db.setAsDefault("Surface Mesh", "None")
db.setAsDefault("Line Geometric", "None")
db.setAsDefault("Surface Geometric", "None")
db.setAsDefault("Isotropic Material", "None")

Generate the mesh

In [ ]:
db.resetMesh()
db.updateMesh()

Define the carriageway by two kerb lines

In [ ]:
overall_length = sum(span_lengths) + abut_thk
# Create lines to represent the carriageway kerbs that can be used for the Vehicle Load Optimiser
curb_line1 = Helpers.create_line([-10,  carriageway_width/2, 0.5], [10 + overall_length,  carriageway_width/2, 0.5])
curb_line2 = Helpers.create_line([-10, -carriageway_width/2, 0.5], [10 + overall_length, -carriageway_width/2, 0.5])
# Set the colour to distinguish them from model lines
curb_line1.setPen(2)
curb_line2.setPen(2)

# Create useful groups 

In [ ]:
group_deck_edge1 = db.createGroup("Deck Edge 1")
group_deck_edge2 = db.createGroup("Deck Edge 2")

# Add lines, start with the first point at either edge of the deck
for iy in [0, len(y_grid)-1]:
    p0 = deck_points[0][iy]
    p1 = deck_points[1][iy]
    # set the group to add lines to
    group = group_deck_edge1 if iy == 0 else group_deck_edge2
    # Find the line that connects the first and second points, i.e. along the x axis and add to the object set
    line = Helpers.get_line_between_points(p0, p1)
    group.add(line)
    # Include all connected collinear lines
    group.addColinearNeighbours(1)

# Loads

In [ ]:
# Get the automatically created loadcase in analysis 1 and add automatic gravity to it
win32.CastTo(db.getLoadset("Loadcase 1", 0).setName("Gravity"), "IFLoadcase").addGravity(True)

In [ ]:
# Create a loadcase for the Parapet loads to be applied to the entire deck surface
parapet_loadcase = db.createLoadcase("Parapet Loads")
# Set the loadcase in the assignment object
assignment = lusas.assignment().setAllDefaults().setLoadset(parapet_loadcase)
# Create a locally distributed load for Parapet loads
parapet_load_attr = db.createLoadingGlobalDistributed("Parapet Loads").setGlobalDistributed("Length", 0.0, 0.0, parapet_load_udl, 0.0, 0.0, 0.0)
# and assign to all the edge lines
parapet_load_attr.assignTo(group_deck_edge1, assignment)
parapet_load_attr.assignTo(group_deck_edge2, assignment)

In [ ]:
# Create a loadcase for the Surfacing loads to be applied to the entire deck surface
surfacing_loadcase = db.createLoadcase("Surfacing")
# Set the loadcase in the assignment object
assignment = lusas.assignment().setAllDefaults().setLoadset(surfacing_loadcase)
# Create a locally distributed load for Surfacing loads
surf_load_attr = db.createLoadingLocalDistributed("Surfacing").setLocalDistributed(0.0, 0.0, surf_load_intensity, "surface")
# and assign to all the surfaces
objs = lusas.newObjectSet().add(deck_surface_attr)
surf_load_attr.assignTo(objs, assignment)

Discrete loads can be be re-used in the model and assigned to a different origin. We'll do that for the footpath loads.<br/>
We'll make a generic function to create a 4 noded discrete patch load the size of the footpath.

In [ ]:
POINT_OFFSET = 9 # Points created for assignment of discrete loads will be offset from the deck for clarity
Z_OFFSET = 1

def create_footpath_load(name:str, load1:float, load2:float)->'IFLoadingDiscretePatch':
    attr = db.createLoadingDiscretePatch(name).setDiscretePatch("surf4", "Z")
    # Add the points of an 4 node patch load in which the load intensity varies across the width
    # The x coordinates are increased by 1 because we'll assign these loads to points created at x=-1 to hold these loads
    attr.addRow(POINT_OFFSET,                0.0,           Z_OFFSET, load1)
    attr.addRow(POINT_OFFSET,                footway_width, Z_OFFSET, load2)
    attr.addRow(POINT_OFFSET+overall_length, footway_width, Z_OFFSET, load2)
    attr.addRow(POINT_OFFSET+overall_length, 0.0,           Z_OFFSET, load1)
    # Explicitly specify how the patch load should be divided up.
    # We know the width of the footway is small so we dont need so many divisions
    # Limiting the divisions speeds up the time to resolve the patch load into nodal loads
    attr.setDivisions(math.ceil(footway_width), 40)

    return attr

In [ ]:
# Create a surfacing load using the generic function
footpath_surf_load_attr = create_footpath_load("Footpath Surface", footpath_surface_load_intensity, footpath_surface_load_intensity)

# Assign this to a new point using the assignment object set up for the deck surfacing loads
footpath_point1 = Helpers.create_point(-POINT_OFFSET, -overall_width/2, 0.0)
footpath_point1.setPen(2)
footpath_surf_load_attr.assignTo(footpath_point1, assignment)
# And again to a 2nd new point
footpath_point2 = Helpers.create_point(-POINT_OFFSET, carriageway_width/2, 0.0)
footpath_point2.setPen(2)
footpath_surf_load_attr.assignTo(footpath_point2, assignment)

# Note that we set the colour of the new points to blue so we can identify that they are not part of the model and only used to locate the discrete loads

Create a settlement load acting only on each support

In [ ]:
# Create the prescribed displacement attribute to represent vertical settlement
settlement_attr = db.createPrescribedDisplacementLoad("Settlement", "Total")
settlement_attr.setDisplacement("W", settlement)

# Create a loadcase for each support settlement
for i, ix in enumerate(base_points.keys()):
    settlement_loadcase = db.createLoadcase(f"Settlement {i+1}")
    # Set the loadcase in the assignment object
    assignment = lusas.assignment().setAllDefaults().setLoadset(settlement_loadcase)
    # Assign to the supported lines
    points = base_points[ix]
    for iy in range(1, len(y_grid)-2):
        line = Helpers.get_line_between_points(points[iy], points[iy+1])
        settlement_attr.assignTo(line, assignment)

# Variable Loads

Create temperature loads

In [ ]:
# Create a loadcases for temperature
temp_warm_loadcase = db.createLoadcase("Temperature +dT")
temp_cool_loadcase = db.createLoadcase("Temperature -dT")
# Temperature loading warming 
temp_warm_attr = db.createLoadingTemperature("Warming").setTemperature("element", temperature_increase, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
# Temperature loading cooling 
temp_cool_attr = db.createLoadingTemperature("Cooling").setTemperature("element", temperature_decrease, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
# Assign the temperature warming load
assignment = lusas.assignment().setAllDefaults().setLoadset(temp_warm_loadcase)
temp_warm_attr.assignTo("Group", "Deck", assignment)
# Assign the temperature cooling load
assignment = lusas.assignment().setAllDefaults().setLoadset(temp_cool_loadcase)
temp_cool_attr.assignTo("Group", "Deck", assignment)

In [ ]:
# Create a loadcases for temperature Gradient
temp_warm_grad_loadcase = db.createLoadcase("Temperature +dT Grad")
temp_cool_grad_loadcase = db.createLoadcase("Temperature -dT Grad")
# Temperature loading warming 
temp_warm_attr = db.createLoadingTemperatureProfile("Warming Gradient").setTopTemperature(temp_max).setBottomTemperature(temp_min)
# Temperature loading cooling 
temp_cool_attr = db.createLoadingTemperatureProfile("Cooling Gradient").setTopTemperature(temp_min).setBottomTemperature(temp_max)
# Assign the temperature warming load
assignment = lusas.assignment().setAllDefaults().setLoadset(temp_warm_grad_loadcase)
temp_warm_attr.assignTo("Group", "Deck", assignment)
# Assign the temperature cooling load
assignment = lusas.assignment().setAllDefaults().setLoadset(temp_cool_grad_loadcase)
temp_cool_attr.assignTo("Group", "Deck", assignment)

Wind loads

In [ ]:
# Create a loadcases for wind
wind_north_loadcase = db.createLoadcase("Wind north")
wind_south_loadcase = db.createLoadcase("Wind south")

# Create a globally distributed load for Wind loads
wind_load_north_attr = db.createLoadingGlobalDistributed("Wind load north").setGlobalDistributed("Length", 0.0, wind_udl, 0.0, wind_mom, 0.0, 0.0)
wind_load_south_attr = db.createLoadingGlobalDistributed("Wind load south").setGlobalDistributed("Length", 0.0, -wind_udl, 0.0, -wind_mom, 0.0, 0.0)

# and assign to all the edge lines
wind_load_north_attr.assignTo(group_deck_edge1, lusas.assignment().setAllDefaults().setLoadset(wind_north_loadcase))
wind_load_south_attr.assignTo(group_deck_edge2, lusas.assignment().setAllDefaults().setLoadset(wind_south_loadcase))

Pedestrian live loads

In [ ]:
# We can reuse the discrete patch load function to create pedestrian live loads 
ped_live_load_attr = create_footpath_load("Pedestrian Live load", pedestrian_live_load, pedestrian_live_load)

# Create a loadcase for the pedestrian live loads
ped_loads_loadcase = db.createLoadcase("Pedestrian live loads")

# Redefine the assignment object for the discrete load assignment as we did for the surfacing loads
assignment = lusas.assignment().setAllDefaults().setLoadset(ped_loads_loadcase)
assignment.setLoadMoving("Exclude All Load")
assignment.setSearchAssignType("area")

# Again, assign the same load to the two dedicated points
ped_live_load_attr.assignTo(footpath_point1, assignment)
ped_live_load_attr.assignTo(footpath_point2, assignment)

Breaking load

In [ ]:

# Create a discrete patch load with the loads acting in the X direction.
# Patch loads are projected onto the structure in the direction of their normal. Which will be the Z axis in this case.
breaking_load_attr = db.createLoadingDiscretePatch("Braking load 1").setDiscretePatch("surf4", "X")
# Add the points of an 4 node patch load
breaking_load_attr.addRow(POINT_OFFSET,                -carriageway_width/2, Z_OFFSET, breaking_load_intensity)
breaking_load_attr.addRow(POINT_OFFSET,                 carriageway_width/2, Z_OFFSET, breaking_load_intensity)
breaking_load_attr.addRow(POINT_OFFSET+overall_length,  carriageway_width/2, Z_OFFSET, breaking_load_intensity)
breaking_load_attr.addRow(POINT_OFFSET+overall_length, -carriageway_width/2, Z_OFFSET, breaking_load_intensity)

# Create a loadcase for the braking loads
breaking_loads_loadcase1 = db.createLoadcase("Braking load +ve")
breaking_loads_loadcase2 = db.createLoadcase("Braking load -ve")

# Redefine the assignment object for the discrete load assignment as we did for the surfacing loads
assignment = lusas.assignment().setAllDefaults().setLoadset(breaking_loads_loadcase1)
assignment.setLoadMoving("Exclude All Load")
assignment.setSearchAssignType("area")

# Assign to the point at the origin
point0 = Helpers.create_point(-POINT_OFFSET, 0.0, 0.0)
point0.setPen(2)
breaking_load_attr.assignTo(point0, assignment)

# Assign the same load with a negative load factor for the opposite direction
assignment.setLoadFactor(-1)
assignment.setLoadset(breaking_loads_loadcase2)
breaking_load_attr.assignTo(point0, assignment)

Horizontal loads

In [ ]:
# # Create the concentrated load attribute
# horiz_load_pos_attr = db.createLoadingConcentrated("Horizontal +ve")
# horiz_load_pos_attr.setConcentrated(0.0, horiz_load, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
# horiz_load_neg_attr = db.createLoadingConcentrated("Horizontal -ve")
# horiz_load_neg_attr.setConcentrated(0.0, -horiz_load, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)

# # Assign positive and negative horizontal loads to each pier
# count = 1
# for ix in pier_indices_X:
#     horiz_pos_loadcase = db.createLoadcase(f"Horizontal +ve {count}")
#     horiz_neg_loadcase = db.createLoadcase(f"Horizontal -ve {count}")
#     # Assign the lateral load to the central column of each pier
#     deck_point = deck_points[ix][pier_indices_Y[1]]
#     assignment = lusas.assignment().setAllDefaults().setLoadset(horiz_pos_loadcase)
#     horiz_load_pos_attr.assignTo(deck_point, assignment)
#     assignment = lusas.assignment().setAllDefaults().setLoadset(horiz_neg_loadcase)
#     horiz_load_neg_attr.assignTo(deck_point, assignment)
#     count+=1

# Load Groups

In [ ]:
# Create envelopes for each load group. They will be grouped in a subfolder in the treeview
GROUP_FOLDER = "Load Groups"

In [ ]:
''' Create some intermediate load groups to help with later design combinations '''

# Get all the loadcases in the model
loadcases = db.getLoadsets("Loadcase")

# Envelope Settlement cases
settlement_env = db.createEnvelope("Settlement Env")
settlement_env.setTreeLocation(GROUP_FOLDER)
for lc in loadcases:
    if "Settlement" in lc.getName():
        settlement_env.addEntry(lc)

# Envelope +/- Temperature cases
temp_annual_env = db.createEnvelope("Temp Annual Env")
temp_annual_env.setTreeLocation(GROUP_FOLDER)
temp_annual_env.addEntry(temp_warm_loadcase)
temp_annual_env.addEntry(temp_cool_loadcase)

temp_daily_env = db.createEnvelope("Temp Daily Env")
temp_daily_env.setTreeLocation(GROUP_FOLDER)
temp_daily_env.addEntry(temp_warm_grad_loadcase)
temp_daily_env.addEntry(temp_cool_grad_loadcase)

# Then combine seasonal and daily variations. Note both max and min versions of the envelope should be included in the smart combination
temp_comb = db.createCombinationSmart("Temperature")
temp_comb.setTreeLocation(GROUP_FOLDER)
temp_comb.addEntry(1,0,temp_annual_env)
temp_comb.addEntry(1,0,temp_annual_env.getAssocLoadset())
temp_comb.addEntry(1,0,temp_daily_env)
temp_comb.addEntry(1,0,temp_daily_env.getAssocLoadset())

# Then combine seasonal and daily variations. Note both max and min versions of the envelope should be included in the smart combination
wind_env = db.createEnvelope("Wind Env")
wind_env.setTreeLocation(GROUP_FOLDER)
wind_env.addEntry(wind_north_loadcase)
wind_env.addEntry(wind_south_loadcase)

# Envelope Braking Loadcases
braking_env = db.createEnvelope("Braking Env")
braking_env.setTreeLocation(GROUP_FOLDER)
for lc in loadcases:
    if "Braking" in lc.getName():
        braking_env.addEntry(lc)

# Envelope Horizontal Loadcases
braking_env = db.createEnvelope("Horizontal Env")
braking_env.setTreeLocation(GROUP_FOLDER)
for lc in loadcases:
    if "Horizontal" in lc.getName():
        braking_env.addEntry(lc)

# Influence analysis

In [ ]:
# An influence analysis considers unit loads applied to a group of elements in order to generate influence surfaces.
# We need to specify the group of elements by assigning a search area
# Create a search area attribute
search_area_attr = db.createSearchArea("Deck Slab")
# Assign the search area attribute to the same surfaces
search_area_attr.assignTo("Group", "Deck", lusas.assignment().setAllDefaults())

In [ ]:
# Create a Direct Method Influence Analysis
dmi_analysis = db.createAnalysisDirectMethodInf("Influence Analysis")
# Specify the elements in the search area deck slab
dmi_analysis.setSearchArea("Deck Slab")
dmi_analysis.setSearchAssignType("area")
dmi_analysis.setIsGrillageType(1)
if dmi_grid:
    dmi_analysis.setGridCentreline("X")
    dmi_analysis.setGridWidth(carriageway_width)

dmi_analysis.setDoMatchNodes(dmi_grid)
dmi_analysis.setSelectedResultsGroup("assignments")
dmi_analysis.createElements()

In [ ]:
# Create an influence attribute which is assigned to specific locations in the model.
inf_attr = db.createDirectInfluence("Deck Major Bending Mx")
inf_attr.setResultsTransformGlobal()
inf_attr.setInfluence("Force/Moment - Thick Shell", "Mx")

# Create an inspection point attribute defined parametrically along a the assigned line
inspect_attr = db.createInspectionPoint("Inspection Points")
inspect_attr.setInspection("Line", "Parametric", True)
for x in influence_locations:
    inspect_attr.addLocation(x, 0.0, 0.0)


# Determine the indices of the deck lines between the supports
span_indicies = [2]
for s in range(len(span_lengths)-1):
    span_indicies.append(span_indicies[-1]+3)
# Select the lines to which the inspection point attribute will be assigned
objs = lusas.newObjectSet()
for iy in influence_lines_y:
    for ix in span_indicies:
        p0 = deck_points[ix][iy]
        p1 = deck_points[ix+1][iy]
        for hof in p0.getHOFs():
            if hof.getStartPoint() == p1 or hof.getEndPoint() == p1:
                objs.add(hof)

# Assign the inspection location to the lines
lusas.assignment().setAllDefaults()
inspect_attr.assignTo(objs.getObjects("Lines"))

# Assign the influence attribute to the inspection locations
inf_attr.assignToInspLocation(inspect_attr.getName(), dmi_analysis.getName()) 


#### Creep and shrinkage

In [ ]:
# Analysis
analysis2 = db.createAnalysisStructural("Analysis 2 - Creep & Shrinkage", False)
# Loadcase
loadcase_cs = db.createLoadcase("C & S", analysis2.getName())
loadcase_cs.addGravity(True)
# Assign creep material
assign = lusas.assignment().setAllDefaults().setLoadset(loadcase_cs)
conc_creep_attr.assignTo("Surfaces", assign)

### Transformation Attributes
Tranformation attributes are used to transform results to consistent axes. They are required to be assigned if slab/wall design is carried out

In [ ]:
# Slabs are in global coordinates
res_transform_deck_attr = db.createResultsTransformation("Deck")
res_transform_deck_attr.setTransformation("Global")
res_transform_deck_attr.assignTo("Group", "Deck")

# Walls we'll transform to have x vertical so that consistent Mx diagrams can be created
local_coord_attr = db.createLocalCartesianXZAttr("Piers", -90.0, [0,0,0]).setAxesType("Cartesian")
res_transform_walls_attr = db.createResultsTransformation("Piers")
res_transform_walls_attr.setTransformation("local coords",None,None,False,None,local_coord_attr.getID())
objs = lusas.newObjectSet().add(abut_surface_attr).add(pier_surface_attr)
res_transform_walls_attr.assignTo(objs)

# Solve the analyses

In [ ]:
# Solve each analysis and open all the available results files
if solve_analyses:
    for analyis in db.getAnalyses():
        analyis.solve(True)
    db.openAllResults(False)

    lusas.view().insertContoursLayer()
    lusas.view().contours().setResultsTransformAttributeType("Results Transformation")
    lusas.view().contours().setResults("Force/Moment - Thick Shell", "Mx")
    lusas.view().contours().setDisplayType("ElementNodal")

In [ ]:
lusas.view().insertValuesLayer()
lusas.view().values().setResultsTransformNone()
lusas.view().values().setResults("Reaction", "FZ")

Set the view to be perspective mode

In [ ]:
lusas.view().setProjection(1)
lusas.view().setIsometric()
lusas.view().setScaledToFit(True)

Set the view to display the assigned loading in the attributes layers as well as the geometric "fleshing"

In [ ]:
lusas.view().insertAttributesLayer()
lusas.view().attributes().drawPatchByDefinition(True)
lusas.view().attributes().visualiseAll("Loading")
lusas.view().attributes().visualiseAll("Geometric")